Test1 to retrive spartpart data and calculate and send back  
Original Author     : Michael Brandon  
Original Modified   : 2024-12-24  
Modified By         : Unknown Player  
Modified Date       : 2024-12-24  

In [45]:
# PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.api import SimpleExpSmoothing, ExponentialSmoothing
from tabulate import tabulate
import os
import json
import sys
import requests
import math
from datetime import datetime


# Set Display Width Longer
# pd.options.display.max_colwidth = 100 # 100 for long width

In [46]:
# Function to print a message with log
def log_message(message):
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"{current_time} - {message}")

# Write to the log file
log_message("Python Spareparts Forecast Program Begins")

2024-12-31 15:08:27 - Python Spareparts Forecast Program Begins


In [47]:
# Retrive JSON Data From API.
url = "http://172.16.1.59:18080/v1/web/test1"

# Fetch JSON data from the API == ALL BRANCH
response = requests.get(url, params= { "all-branch": True })
response.raise_for_status()  # Raise an error if the request fails
df = response.json()  # Parse JSON data

# Convert JSON to Pandas DataFrame
data_all = pd.DataFrame(df['data'])

# Fetch JSON data from the API = SINGLE BRANCH
response = requests.get(url, params= { "all-branch": False })
response.raise_for_status()  # Raise an error if the request fails
df = response.json()  # Parse JSON data

# Convert JSON to Pandas DataFrame
data_branch = pd.DataFrame(df['data'])

log_message("Finish Retriving API")
# log_message("Retrieved " + str(len(data)) + " data")

display(data_all)
display(data_branch)

data = pd.concat([data_all, data_branch], axis=0).reset_index(drop=True)
display(data)

2024-12-31 15:08:28 - Finish Retriving API


,period,branch,agency,partno,D,FD
0,2024-12-01,ALL,08,47585451NP,"[22, 29, 11, 28, 38, 18, 35, 16, 37, 33, 38, 7...",0.0
1,2024-12-01,ALL,15,5181823,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]",0.0


,period,branch,agency,partno,D,FD
0,2024-12-01,21,08,47585451NP,"[0, 6, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0]",0.0
1,2024-12-01,22,08,47585451NP,"[0, 2, 0, 0, 0, 0, 10, 0, 5, 0, 0, 0, 2]",0.0
2,2024-12-01,23,08,47585451NP,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 4]",0.0
3,2024-12-01,25,08,47585451NP,"[2, 0, 0, 0, 0, 4, 0, 0, 0, 2, 0, 2, 0]",0.0
4,2024-12-01,27,08,47585451NP,"[0, 0, 0, 0, 0, 4, 0, 2, 2, 0, 2, 0, 0]",0.0
5,2024-12-01,28,08,47585451NP,"[2, 2, 1, 4, 6, 4, 2, 6, 4, 10, 8, 22, 22]",0.0
6,2024-12-01,30,08,47585451NP,"[0, 7, 2, 6, 4, 0, 4, 0, 8, 0, 4, 6, 4]",0.0
7,2024-12-01,30,15,5181823,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0
8,2024-12-01,32,08,47585451NP,"[0, 0, 0, 0, 4, 0, 2, 0, 0, 0, 0, 0, 0]",0.0
9,2024-12-01,37,08,47585451NP,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]",0.0


,period,branch,agency,partno,D,FD
0,2024-12-01,ALL,08,47585451NP,"[22, 29, 11, 28, 38, 18, 35, 16, 37, 33, 38, 7...",0.0
1,2024-12-01,ALL,15,5181823,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]",0.0
2,2024-12-01,21,08,47585451NP,"[0, 6, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0]",0.0
3,2024-12-01,22,08,47585451NP,"[0, 2, 0, 0, 0, 0, 10, 0, 5, 0, 0, 0, 2]",0.0
4,2024-12-01,23,08,47585451NP,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 4]",0.0
5,2024-12-01,25,08,47585451NP,"[2, 0, 0, 0, 0, 4, 0, 0, 0, 2, 0, 2, 0]",0.0
6,2024-12-01,27,08,47585451NP,"[0, 0, 0, 0, 0, 4, 0, 2, 2, 0, 2, 0, 0]",0.0
7,2024-12-01,28,08,47585451NP,"[2, 2, 1, 4, 6, 4, 2, 6, 4, 10, 8, 22, 22]",0.0
8,2024-12-01,30,08,47585451NP,"[0, 7, 2, 6, 4, 0, 4, 0, 8, 0, 4, 6, 4]",0.0
9,2024-12-01,30,15,5181823,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0


In [48]:
# Retrive Data from json
if 1 == 0:
    with open('from_api.json', 'r') as file:
        jsonData = json.load(file)
    jsonData = jsonData['data'][:5]
    data = pd.DataFrame(jsonData)

In [49]:
# Add Metric to the data
log_message("Begin Calculating Data")

# get mean and standart deviation of first 12 data
data['mean_12'] = data['D'].apply(lambda x: np.mean(x[:12]))
data['std_12'] = data['D'].apply(lambda x: np.std(x[:12], ddof=1))

# get upper bound from mean and std
data['ub'] = data['mean_12'] + 1.5 * data['std_12']

# limit the original data to upper bound
data['clipped_d'] = data.apply(lambda row: np.clip(row['D'][:12], 0, row['ub']).tolist(), axis=1)
# display(data)

2024-12-31 15:08:28 - Begin Calculating Data


In [50]:
# Calculate Simple Moving Average ?? this only calculate mean of data, not SMA
def calculate_ma(list):
    oldData = []
    newData = []
    for i in list:
        # store calculated data to old list
        oldData.append(i)
        newData.append(np.mean(oldData))
    return newData

data['ma'] = data['clipped_d'].apply(calculate_ma)
data['ma_result'] = data['ma'].apply(lambda x: x[-1:])
# data['ma_result'] = data['clipped_d'].apply(lambda x: np.mean(x))
# display(data)


In [51]:
# Calculate Exponential Weighted Moving Average (EWMA)
def ewma(list, alpha):
    df = pd.DataFrame(list)
    df['ewma'] = df.ewm(alpha=alpha, adjust=False).mean()
    return df['ewma'].tolist()

ewma_alpha = 0.3
data['ewma'] = data['clipped_d'].apply(lambda x: ewma(x, ewma_alpha))
data['ewma_result'] = data['ewma'].apply(lambda x: x[-1:])
# display(data['ewma'][0])
# display(data)

In [52]:
#  Calculate Linear Regression
def lr(x):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    model =  LinearRegression()
    model.fit(df[['x']], df['y'])
    df.loc[len(df), 'x'] = len(df) + 1
    return model.predict(df[['x']])

data['lr'] = data['clipped_d'].apply(lambda x: lr(x))
data['lr_result'] = data['lr'].apply(lambda x: x[-1:])
# display(data)


In [53]:
# Calculate Polynomial Regression
def pr(x, pr_degree):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)

    X = df[['x']]  # Independent variable (reshape to 2D array)
    y = df['y']    # Dependent variable

    poly = PolynomialFeatures(degree=pr_degree)  # Create polynomial features
    X_poly = poly.fit_transform(X)  # Transform input features
    poly_model = LinearRegression()  # Initialize linear regression model
    poly_model.fit(X_poly, y)  # Fit polynomial model

    df.loc[len(df), 'x'] = len(df) + 1
    X_all_poly = poly.transform(df[['x']])
    return poly_model.predict(X_all_poly)  

data['pr2'] = data['clipped_d'].apply(lambda x: pr(x, 2))
data['pr2_result'] = data['pr2'].apply(lambda x: x[-1:])
data['pr3'] = data['clipped_d'].apply(lambda x: pr(x, 3))
data['pr3_result'] = data['pr3'].apply(lambda x: x[-1:])
# display(data)


In [54]:
# Calculate Single Exponential Smoothing
def ses(x, alpha):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = SimpleExpSmoothing(df['y']).fit(smoothing_level=alpha, optimized=False).fittedvalues
    return new_data.tolist()

data['ses'] = data['clipped_d'].apply(lambda x: ses(x, 0.8))
data['ses_result'] = data['ses'].apply(lambda x: x[-1:])
# display(data)

In [55]:
# Calculate Double Exponential Smoothing
def des(x, alpha, beta):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = ExponentialSmoothing(df['y'], trend='add', seasonal=None).fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False).fittedvalues
    return new_data.tolist()

data['des'] = data['clipped_d'].apply(lambda x: des(x, 0.8, 0.3))
data['des_result'] = data['des'].apply(lambda x: x[-1:])
# display(data)

In [56]:
# calculate R2 Score and RMSE
log_message("Calculating R2 and RMSE Score")

def metric(x):
    period_lenght = len(x['clipped_d'])
    df = pd.DataFrame()
    df['period'] = range(1, period_lenght + 1)
    df['qty'] = x['clipped_d'][:period_lenght]
    df['ma'] = x['ma'][:period_lenght]
    df['ewma'] = x['ewma'][:period_lenght]
    df['lr'] = x['lr'][:period_lenght]
    df['pr2'] = x['pr2'][:period_lenght]
    df['pr3'] = x['pr3'][:period_lenght]
    df['ses'] = x['ses'][:period_lenght]
    df['des'] = x['des'][:period_lenght]
    # display(df)
    
    # df = pd.concat([pd.DataFrame(x['lr']), df], axis=1)
    result = []
    for i in df.iloc[:, -7:]:
        rmse = np.sqrt(mean_squared_error(df['qty'], df[i]))  # Calculate RMSE
        r2 = r2_score(df['qty'], df[i])  # Calculate R2
        result.append({'model': i, 'RMSE': rmse, 'R2': r2})
        
    # display(result)
    # df_result = pd.DataFrame()

    
    return result


data['metric'] = data.apply(lambda x: metric(x), axis=1)

# display(data['metric'][1])
# display(data)

2024-12-31 15:08:28 - Calculating R2 and RMSE Score


In [57]:
# BEST MODEL SELECTION
def best_select(x, key):
    return max(x, key=lambda x: x['R2'])[key]

def best_number(x):
    return x[x['best_model']][-1]

data['best_model'] = data['metric'].apply(lambda x: best_select(x, 'model'))
data['best_r2'] = data['metric'].apply(lambda x: best_select(x, 'R2'))
data['best_value'] = data.apply(lambda x: best_number(x), axis=1)
data['FD'] = round(data['best_value'])
display(data)


,period,branch,agency,partno,D,FD,mean_12,std_12,ub,clipped_d,...,pr3,pr3_result,ses,ses_result,des,des_result,metric,best_model,best_r2,best_value
0,2024-12-01,ALL,08,47585451NP,"[22, 29, 11, 28, 38, 18, 35, 16, 37, 33, 38, 7...",70.0,31.916667,17.164754,57.663798,"[22.0, 29.0, 11.0, 28.0, 38.0, 18.0, 35.0, 16....",...,"[20.560691749496904, 24.135738041335756, 25.83...",[69.78405392051499],"[22.0, 22.0, 27.6, 14.32, 25.264000000000003, ...",[53.534653808755735],"[21.763636363636355, 23.106424242424247, 30.38...",[60.86968659615782],"[{'model': 'ma', 'RMSE': 11.206657787037711, '...",pr3,0.571893,69.784054
1,2024-12-01,ALL,15,5181823,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]",0.0,0.250000,0.452267,0.928401,"[0.0, 0.0, 0.0, 0.9284005252999682, 0.0, 0.0, ...",...,"[9.409140133698202e-15, 0.09089235912028129, 0...",[0.28133349251511375],"[0.0, 0.0, 0.0, 0.0, 0.7427204202399746, 0.148...",[0.14973433808465467],"[0.135040076407268, 0.005851736644314969, -0.0...",[0.1258617227636162],"[{'model': 'ma', 'RMSE': 0.3688366642940196, '...",ewma,0.342701,0.277893
2,2024-12-01,21,08,47585451NP,"[0, 6, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0]",1.0,0.916667,1.781640,3.589127,"[0.0, 3.5891272284983007, 2.0, 2.0, 0.0, 0.0, ...",...,"[1.2694366810835163, 1.6890108192650681, 1.785...",[1.323934613555545],"[0.0, 0.0, 2.871301782798641, 2.17426035655972...",[0.0002612092213025583],"[1.8986551937449585, -0.3070000901709382, 3.05...",[-0.1085831280028173],"[{'model': 'ma', 'RMSE': 0.9629892088464685, '...",pr3,0.455780,1.323935
3,2024-12-01,22,08,47585451NP,"[0, 2, 0, 0, 0, 0, 10, 0, 5, 0, 0, 0, 2]",1.0,1.416667,3.088346,6.049185,"[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 6.0491851256398...",...,"[0.741067429454173, 0.274599672488141, 0.24903...",[-3.3371432678573054],"[0.0, 0.0, 1.6, 0.31999999999999995, 0.0639999...",[0.03354875523216378],"[0.23725791137526975, 0.22776759492025928, 2.2...",[-0.5159956741405223],"[{'model': 'ma', 'RMSE': 1.9121713623777699, '...",ewma,0.362431,0.836454
4,2024-12-01,23,08,47585451NP,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 4]",0.0,0.333333,0.778499,1.501082,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.501...",...,"[0.23643412172790812, -0.12346559845000848, -0...",[-0.7429596538300299],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[0.05764153918370368],"[-0.354801140808782, 0.15975148801544142, 0.22...",[-0.10641698349306405],"[{'model': 'ma', 'RMSE': 0.5291107493423707, '...",ewma,0.448001,0.375120
5,2024-12-01,25,08,47585451NP,"[2, 0, 0, 0, 0, 4, 0, 0, 0, 2, 0, 2, 0]",1.0,0.833333,1.337116,2.839007,"[2.0, 0.0, 0.0, 0.0, 0.0, 2.839007103597898, 0...",...,"[1.22530330072992, 0.8033132355735317, 0.51994...",[1.5281610421335499],"[2.0, 2.0, 0.3999999999999999, 0.0799999999999...",[1.6641453981237042],"[0.606473243898029, 2.072947173902248, 0.26873...",[1.9619252758701438],"[{'model': 'ma', 'RMSE': 0.950142373340223, 'R...",ewma,0.254245,1.033748
6,2024-12-01,27,08,47585451NP,"[0, 0, 0, 0, 0, 4, 0, 2, 2, 0, 2, 0, 0]",1.0,0.833333,1.337116,2.839007,"[0.0, 0.0, 0.0, 0.0, 0.0, 2.839007103597898, 0...",...,"[-0.03476995934338811, -0.09112565886533441, 0...",[-0.9364854225129227],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.2712056828783...",[0.33550535716370417],"[-0.048072210647425534, 0.16458353753861155, 0...",[0.11228948444195053],"[{'model': 'ma', 'RMSE': 0.9869020244704817, '...",ewma,0.415660,0.870062
7,2024-12-01,28,08,47585451NP,"[2, 2, 1, 4, 6, 4, 2, 6, 4, 10, 8, 22, 22]",19.0,5.916667,5.728054,14.508748,"[2.0, 2.0, 1.0, 4.0, 6.0, 4.0, 2.0, 6.0, 4.0, ...",...,"[1.2006961363002056, 2.434616966815212, 3.1372...",[18.516714660056685],"[2.0, 2.0, 2.0, 1.2, 3.44, 5.488, 4.2976, 2.45...",[13.24106553940417],"[1.290909090909091, 2.6526060606060606, 2.7683...",[15.62268266552892],"[{'model': 'ma', 'RMSE': 3.5791299698686525, '...",pr3,0.815171,18.516715
8,2024-12-01,30,08,47585451NP,"[0, 7, 2, 6, 4, 0, 4, 0, 8, 0, 4, 6, 4]",4.0,3.416667,2.968267,7.869066,"[0.0, 7.0, 2.0, 6.0, 4.0, 0.0, 4.0, 0.0, 7.869...",...,"[1.822389238591541, 3.3768920964925306,

In [58]:
# Send Data Back To API

# API endpoint
url = "http://172.16.1.59:18080/v1/web/test1-post"

data2 = data[['period', 'branch', 'agency', 'partno', 'FD', 'mean_12', 'std_12', 'ub']]
json2 = data2.to_dict(orient='records')

log_message("Start Sending " + str(len(data2)) + " To API")

# Send POST request
response = requests.post(url, json=json2)

log_message(f"Send API Complete")
log_message(f"Status Code: {response.status_code}")
log_message(f"Response Body: {response.text}")
print(str(response.json().get("success", "No status key found")))

2024-12-31 15:08:29 - Start Sending 27 To API
2024-12-31 15:08:29 - Send API Complete
2024-12-31 15:08:29 - Status Code: 200
2024-12-31 15:08:29 - Response Body: <BR>Unable to run web object file<BR>

<!-- Generated by Webspeed: http://www.webspeed.com/ -->



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [19]:
# Convert DataFrame to JSON and write to a file

# Specify the target directory where you want to save the file
# target_folder = 'output/'  # Update this path

# # Ensure the directory exists (create if it doesn't)
# os.makedirs(target_folder, exist_ok=True)

# # Full file path
# file_path = os.path.join(target_folder, 'result.json')

# data.to_json(file_path, orient='records', lines=False)
# print(True)